# Import

In [1]:
from __future__ import division, print_function, absolute_import

import numpy as np
import glob
import csv
import os
import io
import pandas as pd
import tensorflow as tf
import dataset_util
import h5py

from PIL import Image
from collections import namedtuple, OrderedDict
from datetime import datetime
from matplotlib import pyplot as plt
from collections import defaultdict



In [2]:
def cv_img_size(img):
    return tuple(img.shape[1::-1])

def plotCVColorImage(image, size = (800, 800)):
    converted_size = (size[0] / 100, size[1] / 100)
    plt.figure(figsize=converted_size, dpi = 100)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
def plotRGBColorImage(image, size = (800, 800)):
    converted_size = (size[0] / 100, size[1] / 100)
    plt.figure(figsize=converted_size, dpi = 100)
    plt.axis("off")
    plt.imshow(image)
    plt.show()

# Retrieve Labels *

Run this to retrieve all the labels for next stage

In [3]:
image_label_file = '../data/labels/cleaned/RF.csv'

ignore_label_id = [102, 103]

DELIMITER = '|'
QUOTE = "'"

DB_VERSION = "DBVERSION:2.0"
FILES_SEPARATER = "/////FILES/////"
LABEL_SEPARATER = "/////LABEL/////"
LABELS_SEPARTER = "/////LABELS/////"
BOUNDINGBOXES_SEPARTER = "/////BOUNDINGBOXES/////"

partial_dataset = False

In [4]:
files_dictionary = {} # store file names only
files_info_dictionary = {} # store other info
files_inverse_dictionary = {}
label_name_dictionary = {}
labels_dictionary = defaultdict(list)
boundingboxes_dictionary = defaultdict(list)

with open(image_label_file) as csvfile:
    label_reader = csv.reader(csvfile, delimiter = DELIMITER, quotechar = QUOTE)
    current_section = 'BEGIN'

    for row in label_reader:
        if row == [FILES_SEPARATER]:
            current_section = 'FILES'
        elif row == [LABEL_SEPARATER]:
            current_section = 'LABEL'
        elif row == [LABELS_SEPARTER]:
            current_section = 'LABELS'
        elif row == [BOUNDINGBOXES_SEPARTER]:
            current_section = 'BOUNDINGBOXES'

        if len(row) > 1:
            if current_section == 'FILES':
                files_dictionary[int(row[0])] = row[1]
                files_info_dictionary[int(row[0])] = {'name': row[1], 'path': row[2], 'width': int(row[3]), 'height': int(row[4]), 'hash': row[5]}
                files_inverse_dictionary[row[1]] = int(row[0])
            elif current_section == 'LABEL':
                if not int(row[0]) in ignore_label_id:
                    label_name_dictionary[int(row[0])] = {'name': row[1]}
            elif current_section == 'LABELS':
                if not int(row[3]) in ignore_label_id:
                    labels_dictionary[int(row[0])].append( {'x_coor': int(row[1]), 'y_coor': int(row[2]), 'label': int(row[3])} )
            elif current_section == 'BOUNDINGBOXES':
                if not int(row[5]) in ignore_label_id:
                    boundingboxes_dictionary[int(row[0])].append( {'x_coor': int(row[1]), 'y_coor': int(row[2]), 'width': int(row[3]), 'height': int(row[4]), 'label': int(row[5])} )
                
    #print(labels_dictionary)
        

In [5]:
print(files_dictionary)

{1: '1.JPG', 2: '2.JPG', 3: '3.JPG', 4: '4.JPG', 5: '5.JPG', 6: '6.JPG', 7: '7.JPG', 8: '8.JPG', 9: '9.JPG', 10: '10.JPG', 11: '11.JPG', 12: '12.JPG', 13: '13.JPG', 14: '14.JPG', 15: '15.JPG', 16: '16.JPG', 17: '17.JPG', 18: '18.JPG', 19: '19.JPG', 20: '20.JPG', 21: '21.JPG', 22: '22.JPG', 23: '23.JPG', 24: '24.JPG', 25: '25.JPG', 26: '26.JPG', 27: '27.JPG', 28: '28.JPG', 29: '29.JPG', 30: '30.JPG', 31: '38.JPG', 32: '31.JPG', 33: '32.JPG', 34: '33.JPG', 35: '34.JPG', 36: '35.JPG', 37: '36.JPG', 38: '37.JPG', 39: '39.JPG', 40: '40.JPG', 41: '41.JPG', 42: '42.JPG', 43: '43.JPG', 44: '44.JPG', 45: '45.JPG', 46: '46.JPG', 47: '47.JPG', 48: '48.JPG', 49: '49.JPG', 50: '50.JPG', 51: '52.JPG', 52: '53.JPG', 53: '54.JPG', 54: '56.JPG', 55: '57.JPG', 56: '58.JPG', 57: '59.JPG', 58: '60.JPG', 59: '61.JPG', 60: '62.JPG', 61: '63.JPG', 62: '64.JPG', 63: '65.JPG', 64: '66.JPG', 65: '67.JPG', 66: '68.JPG', 67: '69.JPG', 68: '70.JPG', 69: '71.JPG', 70: '72.JPG', 71: '73.JPG', 72: '74.JPG', 73: '75.J

In [6]:
print(label_name_dictionary)

{1: {'name': 'Lactobacillus    '}, 2: {'name': 'Gardnerella      '}, 3: {'name': 'Curved Rod       '}, 4: {'name': 'Unknown A        '}, 5: {'name': 'Coccus           '}, 6: {'name': 'Yeast            '}, 7: {'name': 'Noise - Edge     '}, 8: {'name': 'Noise - Decompose'}}


In [7]:
# Don't run if not wanted to output
# image_label_output_file = '../data/labels/cleaned/Adj_MAC.csv'
# write_dataset_to_file(image_label_output_file, files_info_dictionary, label_name_dictionary, labels_dictionary)

In [8]:
# label stat:
label_count = defaultdict(int)
for file, list_label in boundingboxes_dictionary.items():
    for l in list_label:
        label_count[l['label']] += 1
print(label_count)

defaultdict(<class 'int'>, {2: 2411, 7: 89, 1: 531, 8: 123, 3: 232, 5: 3, 6: 19})


In [8]:
# Put back
o_files_dictionary = files_dictionary

o_files_info_dictionary = files_info_dictionary

o_files_inverse_dictionary = files_inverse_dictionary

o_labels_dictionary = labels_dictionary

o_boundingboxes_dictionary = boundingboxes_dictionary

## Interpretations Output (NOT NECESS)

In [9]:
from interpretation_util import *

In [10]:
DELIMITER = ','
QUOTE = "'"
output_file_path = "../data/interpretations/raw/count.csv"

In [24]:
def number_extract(str_a):
    if isinstance(str_a, str):
        if len(str_a) == 0:
            return 0
        return int(str_a[:str_a.find('.')])
    else:
        return -1

In [ ]:
import operator
import functools
with open(output_file_path, 'w', newline='') as csvfile:
    db_writer = csv.writer(csvfile, delimiter=DELIMITER, quotechar=QUOTE)
    db_writer.writerow(['Image', 'Lactobacillus Count', 'Lactobacillus', 
                        'Gardnerella Count', 'Gardnerella', 
                        'Curved Rods Count', 'Curved Rods', 'Total', 'Interpretation'])
    
    file_names = files_dictionary.keys()
    
    for fid, fname in sorted(files_dictionary.items(), key= lambda item: number_extract(item[1])):
        bacteria_count = [0, 0, 0]
        for label in labels_dictionary[fid]:
            if label['label'] >= 1 and label['label'] <= 3:
                bacteria_count[label['label'] - 1] += 1
        scores = get_nugent_score(bacteria_count[0], bacteria_count[1], bacteria_count[2])
        db_writer.writerow([fname, 
                            bacteria_count[0], 
                            scores[1], 
                            bacteria_count[1], 
                            scores[2],
                            bacteria_count[2], 
                            scores[3],
                            scores[0], 
                            get_nugent_score_interpretation_str(scores[0])])
        

In [29]:
full_interp = "../data/interpretations/raw/merged.csv"
output_file_path = "../data/interpretations/raw/merged_clean.csv"
DELIMITER = ','
QUOTE = "'"

In [33]:
# Merge Ground Truth
with open(output_file_path, 'w', newline='') as csvoutfile:
    with open(full_interp, 'r') as csvinfile:
        db_writer = csv.writer(csvoutfile, delimiter=DELIMITER, quotechar=QUOTE)
        db_reader = csv.reader(csvinfile, delimiter=DELIMITER, quotechar=QUOTE)

        begin = True
        for row in db_reader:
            if begin:
                db_writer.writerow(['Image', 'Lactobacillus', 'Gardnerella', 'Curved Rods', 'Total', 'Interpretation'])
                begin = False
                continue
            scores = [min(int(row[2]), int(row[3])), max(int(row[5]), int(row[6])), max(int(row[8]), int(row[9]))]
            total = sum(scores)
            new_row = [row[0], scores[0], scores[1], scores[2], total, get_nugent_score_interpretation_str(total)]
            print([int(row[2]), int(row[3]), int(row[5]), int(row[6]), int(row[8]), int(row[9])], scores)
            db_writer.writerow(new_row)


[4, 1, 4, 3, 0, 0] [1, 4, 0]
[2, 2, 0, 0, 0, 0] [2, 0, 0]
[0, 0, 0, 0, 0, 0] [0, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[3, 3, 4, 4, 1, 1] [3, 4, 1]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[4, 4, 4, 4, 2, 2] [4, 4, 2]
[4, 4, 4, 4, 2, 2] [4, 4, 2]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[1, 1, 4, 4, 0, 0] [1, 4, 0]
[1, 1, 3, 3, 0, 0] [1, 3, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[2, 1, 0, 0, 0, 0] [1, 0, 0]
[2, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[2, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 2, 0, 0, 0] [1, 2, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[2, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[1, 1, 0, 0, 0, 0] [1, 0, 0]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[4, 4, 3, 4, 1, 1] [4, 4, 1]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[2, 4, 4, 3, 2, 1] [2, 4, 2]
[4, 4, 4, 4, 1, 1] [4, 4, 1]
[4, 4, 4, 3, 1, 0] [4, 4, 1]
[4, 4, 4, 3, 0, 0] [4, 4, 0]
[4, 4, 3, 3, 1

## Image Segmentation

In [7]:
target_w = 160
target_h = 160
image_input_dir = '../data/images/adjusted/labelled/'
image_output_dir = '../data/images/adjusted/segmented/'
image_name_format = '{}_{:04d}_{:04d}{}'

In [8]:
image_input_list = glob.glob(image_input_dir + "*.JPG")

In [60]:
for file_path in image_input_list:
    base_name = os.path.splitext(os.path.basename(file_path))
    img_full = cv2.imread(file_path)
    img_size = cv_img_size(img_full)
    h_pieces = img_size[0] // (target_w // 2) - 1
    v_pieces = img_size[1] // (target_h // 2) - 1
    for j in range(v_pieces):
        for i in range(h_pieces):
            ya = j * target_h // 2
            yb = (j + 2) * target_h // 2
            xa = i * target_w // 2
            xb = (i + 2) * target_w // 2
            crop = img_full[ya: yb, xa: xb]
            cv2.imwrite(image_output_dir + image_name_format.format(base_name[0], xa, ya, base_name[1]), crop)

# Label Segmentation

In [9]:
partial_boxes_enabled = True
cut_off_threshold = 0.5

empty_region_ignore = True

s_o_files_id_map = defaultdict(list)

s_files_dictionary = {} # store file names only
s_files_info_dictionary = {} # store other info
s_files_inverse_dictionary = {}
# s_label_name_dictionary = {}
s_labels_dictionary = defaultdict(list)
s_boundingboxes_dictionary = defaultdict(list)
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0), (255, 255, 255)]

partial_dataset = True

In [10]:
new_id = 0
for fid, fname in files_dictionary.items():
    print(fid, fname)
    base_name = os.path.splitext(fname)
    img_size = (files_info_dictionary[fid]['width'], files_info_dictionary[fid]['height'])
    h_pieces = img_size[0] // (target_w // 2) - 1
    v_pieces = img_size[1] // (target_h // 2) - 1
    
    this_labels_list = labels_dictionary[fid]
    this_boundingboxes_list = boundingboxes_dictionary[fid]
    
    for j in range(v_pieces):
        for i in range(h_pieces):
            new_id += 1
            ya = j * target_h // 2
            yb = (j + 2) * target_h // 2
            xa = i * target_w // 2
            xb = (i + 2) * target_w // 2

            crop_file_name = image_name_format.format(base_name[0], xa, ya, base_name[1])
            s_files_dictionary[new_id] = crop_file_name
            s_files_info_dictionary[new_id] = {'name': crop_file_name, 
                                               'path': os.path.abspath(image_output_dir + crop_file_name), 
                                               'width': target_w, 
                                               'height': target_h, 
                                               'hash': '0'}
            s_files_inverse_dictionary[crop_file_name] = new_id
            s_labels_dictionary[new_id] = [
                {'x_coor': p['x_coor'] - xa, 'y_coor': p['y_coor'] - ya, 'label': p['label']} for p in this_labels_list if 
                    (p['x_coor'] > xa and p['x_coor'] <= xb and p['y_coor'] > ya and p['y_coor'] <= yb)
            ]
            if partial_boxes_enabled:
                s_boundingboxes_dictionary[new_id] = [
                    {'x_coor': x, 'y_coor': y, 'width': w, 'height': h, 'label': r['label']} 
                        for r in this_boundingboxes_list 
                        for (x, y, w, h) in 
                            [(
                                max(r['x_coor'] - xa, 0),
                                max(r['y_coor'] - ya, 0),
                                min(r['x_coor'] - xa + r['width'] - max(r['x_coor'] - xa, 0), target_w - max(r['x_coor'] - xa, 0)),
                                min(r['y_coor'] - ya + r['height'] - max(r['y_coor'] - ya, 0), target_h - max(r['y_coor'] - ya, 0))
                             )]
                        if
                            (xa < r['x_coor'] + r['width'] and r['x_coor'] < xb and 
                             ya < r['y_coor'] + r['height'] and r['y_coor'] < yb and
                             (cut_off_threshold * r['width'] <= w or w == target_w) and
                             (cut_off_threshold * r['height'] <= h or h == target_h)
                            )
                ]
                
#                 s_boundingboxes_dictionary[new_id] = [
#                     {'x_coor': max(r['x_coor'] - xa, 0), 'y_coor': max(r['y_coor'] - ya, 0), 
#                      'width': min(r['x_coor'] - xa + r['width'] - max(r['x_coor'] - xa, 0), target_w - max(r['x_coor'] - xa, 0)), 
#                      'height': min(r['y_coor'] - ya + r['height'] - max(r['y_coor'] - ya, 0), target_h - max(r['y_coor'] - ya, 0)), 
#                      'label': r['label']} 
#                         for r in this_boundingboxes_list if
#                             (xa < r['x_coor'] + r['width'] and r['x_coor'] < xb and 
#                              ya < r['y_coor'] + r['height'] and r['y_coor'] < yb and
#                              cut_off_threshold * r['width'] <= min(r['x_coor'] - xa + r['width'] - max(r['x_coor'] - xa, 0), target_w - max(r['x_coor'] - xa, 0)) and
#                              cut_off_threshold * r['height'] <= min(r['y_coor'] - ya + r['height'] - max(r['y_coor'] - ya, 0), target_h - max(r['y_coor'] - ya, 0))
#                             )
#                 ]
            else:
                s_boundingboxes_dictionary[new_id] = [
                    {'x_coor': r['x_coor'] - xa, 'y_coor': r['y_coor'] - ya, 
                     'width': r['width'], 'height': r['height'], 'label': r['label']} 
                        for r in this_boundingboxes_list if
                            (r['x_coor'] > xa and r['x_coor'] + r['width'] <= xb and 
                             r['y_coor'] > ya and r['y_coor'] + r['height'] <= yb)
                ]
            num = len(s_boundingboxes_dictionary[new_id])
            print(crop_file_name, new_id, num)
            if empty_region_ignore and num == 0:
                new_id -= 1
            else:
                s_o_files_id_map[fid].append(new_id)
            
#             print(crop_file_name)
#             img_crop = cv2.imread(os.path.abspath(image_output_dir + crop_file_name))
#             for r in s_boundingboxes_dictionary[new_id]:
#                 cv2.rectangle(img_crop,
#                     (r['x_coor'], r['y_coor']),
#                     (r['x_coor'] + r['width'], r['y_coor'] + r['height']),
#                     colors[-1] if r['label'] >= len(colors) else colors[r['label'] - 1])
#                 print((r['x_coor'], r['y_coor']),
#                     (r['x_coor'] + r['width'], r['y_coor'] + r['height']))
#             plotCVColorImage(img_crop, (target_w * 2 , target_h * 2))

#     print(this_labels_list)
#     print("--------------------")
#     print(s_labels_dictionary)
#     print("////////////////////")
#     print(this_labels_list)
#     print("--------------------")
#     print(s_boundingboxes_dictionary)
#     if new_id > 200:
#         break

1 1.JPG
1_0000_0000.JPG 1 0
1_0080_0000.JPG 1 2
1_0160_0000.JPG 2 2
1_0240_0000.JPG 3 0
1_0320_0000.JPG 3 0
1_0400_0000.JPG 3 0
1_0480_0000.JPG 3 1
1_0560_0000.JPG 4 3
1_0640_0000.JPG 5 2
1_0720_0000.JPG 6 1
1_0800_0000.JPG 7 1
1_0880_0000.JPG 8 0
1_0960_0000.JPG 8 0
1_1040_0000.JPG 8 0
1_1120_0000.JPG 8 0
1_0000_0080.JPG 8 0
1_0080_0080.JPG 8 1
1_0160_0080.JPG 9 1
1_0240_0080.JPG 10 0
1_0320_0080.JPG 10 1
1_0400_0080.JPG 11 1
1_0480_0080.JPG 12 0
1_0560_0080.JPG 12 1
1_0640_0080.JPG 13 1
1_0720_0080.JPG 14 1
1_0800_0080.JPG 15 1
1_0880_0080.JPG 16 0
1_0960_0080.JPG 16 0
1_1040_0080.JPG 16 0
1_1120_0080.JPG 16 1
1_0000_0160.JPG 17 0
1_0080_0160.JPG 17 0
1_0160_0160.JPG 17 1
1_0240_0160.JPG 18 2
1_0320_0160.JPG 19 2
1_0400_0160.JPG 20 1
1_0480_0160.JPG 21 0
1_0560_0160.JPG 21 2
1_0640_0160.JPG 22 3
1_0720_0160.JPG 23 3
1_0800_0160.JPG 24 2
1_0880_0160.JPG 25 0
1_0960_0160.JPG 25 1
1_1040_0160.JPG 26 2
1_1120_0160.JPG 27 2
1_0000_0240.JPG 28 1
1_0080_0240.JPG 29 0
1_0160_0240.JPG 29 1
1_

3_0800_0720.JPG 209 4
3_0880_0720.JPG 210 2
3_0960_0720.JPG 211 0
3_1040_0720.JPG 211 0
3_1120_0720.JPG 211 1
3_0000_0800.JPG 212 0
3_0080_0800.JPG 212 0
3_0160_0800.JPG 212 0
3_0240_0800.JPG 212 0
3_0320_0800.JPG 212 0
3_0400_0800.JPG 212 0
3_0480_0800.JPG 212 1
3_0560_0800.JPG 213 2
3_0640_0800.JPG 214 2
3_0720_0800.JPG 215 1
3_0800_0800.JPG 216 0
3_0880_0800.JPG 216 0
3_0960_0800.JPG 216 0
3_1040_0800.JPG 216 0
3_1120_0800.JPG 216 1
4 4.JPG
4_0000_0000.JPG 217 0
4_0080_0000.JPG 217 0
4_0160_0000.JPG 217 0
4_0240_0000.JPG 217 0
4_0320_0000.JPG 217 0
4_0400_0000.JPG 217 0
4_0480_0000.JPG 217 0
4_0560_0000.JPG 217 0
4_0640_0000.JPG 217 0
4_0720_0000.JPG 217 0
4_0800_0000.JPG 217 0
4_0880_0000.JPG 217 0
4_0960_0000.JPG 217 0
4_1040_0000.JPG 217 0
4_1120_0000.JPG 217 0
4_0000_0080.JPG 217 0
4_0080_0080.JPG 217 0
4_0160_0080.JPG 217 0
4_0240_0080.JPG 217 1
4_0320_0080.JPG 218 1
4_0400_0080.JPG 219 0
4_0480_0080.JPG 219 1
4_0560_0080.JPG 220 1
4_0640_0080.JPG 221 0
4_0720_0080.JPG 221 0
4_

7_1120_0720.JPG 426 0
7_0000_0800.JPG 426 2
7_0080_0800.JPG 427 1
7_0160_0800.JPG 428 1
7_0240_0800.JPG 429 3
7_0320_0800.JPG 430 4
7_0400_0800.JPG 431 3
7_0480_0800.JPG 432 1
7_0560_0800.JPG 433 1
7_0640_0800.JPG 434 3
7_0720_0800.JPG 435 3
7_0800_0800.JPG 436 2
7_0880_0800.JPG 437 0
7_0960_0800.JPG 437 0
7_1040_0800.JPG 437 0
7_1120_0800.JPG 437 0
8 8.JPG
8_0000_0000.JPG 437 3
8_0080_0000.JPG 438 7
8_0160_0000.JPG 439 5
8_0240_0000.JPG 440 0
8_0320_0000.JPG 440 0
8_0400_0000.JPG 440 0
8_0480_0000.JPG 440 0
8_0560_0000.JPG 440 0
8_0640_0000.JPG 440 1
8_0720_0000.JPG 441 1
8_0800_0000.JPG 442 0
8_0880_0000.JPG 442 1
8_0960_0000.JPG 443 1
8_1040_0000.JPG 444 2
8_1120_0000.JPG 445 2
8_0000_0080.JPG 446 4
8_0080_0080.JPG 447 6
8_0160_0080.JPG 448 4
8_0240_0080.JPG 449 0
8_0320_0080.JPG 449 0
8_0400_0080.JPG 449 0
8_0480_0080.JPG 449 1
8_0560_0080.JPG 450 1
8_0640_0080.JPG 451 1
8_0720_0080.JPG 452 1
8_0800_0080.JPG 453 0
8_0880_0080.JPG 453 1
8_0960_0080.JPG 454 1
8_1040_0080.JPG 455 1
8_

10_0160_0800.JPG 793 2
10_0240_0800.JPG 794 2
10_0320_0800.JPG 795 0
10_0400_0800.JPG 795 0
10_0480_0800.JPG 795 0
10_0560_0800.JPG 795 1
10_0640_0800.JPG 796 1
10_0720_0800.JPG 797 0
10_0800_0800.JPG 797 0
10_0880_0800.JPG 797 1
10_0960_0800.JPG 798 2
10_1040_0800.JPG 799 1
10_1120_0800.JPG 800 2
11 11.JPG
11_0000_0000.JPG 801 0
11_0080_0000.JPG 801 1
11_0160_0000.JPG 802 1
11_0240_0000.JPG 803 0
11_0320_0000.JPG 803 0
11_0400_0000.JPG 803 1
11_0480_0000.JPG 804 1
11_0560_0000.JPG 805 0
11_0640_0000.JPG 805 0
11_0720_0000.JPG 805 0
11_0800_0000.JPG 805 0
11_0880_0000.JPG 805 0
11_0960_0000.JPG 805 0
11_1040_0000.JPG 805 0
11_1120_0000.JPG 805 0
11_0000_0080.JPG 805 0
11_0080_0080.JPG 805 0
11_0160_0080.JPG 805 0
11_0240_0080.JPG 805 0
11_0320_0080.JPG 805 0
11_0400_0080.JPG 805 1
11_0480_0080.JPG 806 1
11_0560_0080.JPG 807 1
11_0640_0080.JPG 808 1
11_0720_0080.JPG 809 0
11_0800_0080.JPG 809 0
11_0880_0080.JPG 809 0
11_0960_0080.JPG 809 0
11_1040_0080.JPG 809 0
11_1120_0080.JPG 809 0
1

14_0400_0800.JPG 976 0
14_0480_0800.JPG 976 0
14_0560_0800.JPG 976 0
14_0640_0800.JPG 976 0
14_0720_0800.JPG 976 0
14_0800_0800.JPG 976 0
14_0880_0800.JPG 976 0
14_0960_0800.JPG 976 0
14_1040_0800.JPG 976 0
14_1120_0800.JPG 976 0
15 15.JPG
15_0000_0000.JPG 976 0
15_0080_0000.JPG 976 0
15_0160_0000.JPG 976 0
15_0240_0000.JPG 976 0
15_0320_0000.JPG 976 0
15_0400_0000.JPG 976 0
15_0480_0000.JPG 976 0
15_0560_0000.JPG 976 0
15_0640_0000.JPG 976 0
15_0720_0000.JPG 976 0
15_0800_0000.JPG 976 0
15_0880_0000.JPG 976 0
15_0960_0000.JPG 976 0
15_1040_0000.JPG 976 0
15_1120_0000.JPG 976 0
15_0000_0080.JPG 976 0
15_0080_0080.JPG 976 0
15_0160_0080.JPG 976 0
15_0240_0080.JPG 976 0
15_0320_0080.JPG 976 0
15_0400_0080.JPG 976 0
15_0480_0080.JPG 976 0
15_0560_0080.JPG 976 0
15_0640_0080.JPG 976 0
15_0720_0080.JPG 976 0
15_0800_0080.JPG 976 0
15_0880_0080.JPG 976 0
15_0960_0080.JPG 976 0
15_1040_0080.JPG 976 0
15_1120_0080.JPG 976 0
15_0000_0160.JPG 976 0
15_0080_0160.JPG 976 0
15_0160_0160.JPG 976 0
1

...

112_0160_0400.JPG 6392 3
112_0240_0400.JPG 6393 3
112_0320_0400.JPG 6394 2
112_0400_0400.JPG 6395 1
112_0480_0400.JPG 6396 0
112_0560_0400.JPG 6396 1
112_0640_0400.JPG 6397 3
112_0720_0400.JPG 6398 2
112_0800_0400.JPG 6399 0
112_0880_0400.JPG 6399 0
112_0960_0400.JPG 6399 0
112_1040_0400.JPG 6399 0
112_1120_0400.JPG 6399 0
112_0000_0480.JPG 6399 0
112_0080_0480.JPG 6399 0
112_0160_0480.JPG 6399 2
112_0240_0480.JPG 6400 4
112_0320_0480.JPG 6401 3
112_0400_0480.JPG 6402 1
112_0480_0480.JPG 6403 0
112_0560_0480.JPG 6403 1
112_0640_0480.JPG 6404 3
112_0720_0480.JPG 6405 2
112_0800_0480.JPG 6406 0
112_0880_0480.JPG 6406 0
112_0960_0480.JPG 6406 1
112_1040_0480.JPG 6407 1
112_1120_0480.JPG 6408 0
112_0000_0560.JPG 6408 0
112_0080_0560.JPG 6408 0
112_0160_0560.JPG 6408 0
112_0240_0560.JPG 6408 3
112_0320_0560.JPG 6409 4
112_0400_0560.JPG 6410 1
112_0480_0560.JPG 6411 0
112_0560_0560.JPG 6411 0
112_0640_0560.JPG 6411 0
112_0720_0560.JPG 6411 0
112_0800_0560.JPG 6411 1
112_0880_0560.JPG 6412 1


114_0320_0400.JPG 6521 1
114_0400_0400.JPG 6522 1
114_0480_0400.JPG 6523 0
114_0560_0400.JPG 6523 0
114_0640_0400.JPG 6523 0
114_0720_0400.JPG 6523 0
114_0800_0400.JPG 6523 2
114_0880_0400.JPG 6524 2
114_0960_0400.JPG 6525 0
114_1040_0400.JPG 6525 1
114_1120_0400.JPG 6526 1
114_0000_0480.JPG 6527 0
114_0080_0480.JPG 6527 0
114_0160_0480.JPG 6527 0
114_0240_0480.JPG 6527 0
114_0320_0480.JPG 6527 1
114_0400_0480.JPG 6528 1
114_0480_0480.JPG 6529 0
114_0560_0480.JPG 6529 0
114_0640_0480.JPG 6529 0
114_0720_0480.JPG 6529 0
114_0800_0480.JPG 6529 0
114_0880_0480.JPG 6529 0
114_0960_0480.JPG 6529 0
114_1040_0480.JPG 6529 1
114_1120_0480.JPG 6530 1
114_0000_0560.JPG 6531 0
114_0080_0560.JPG 6531 0
114_0160_0560.JPG 6531 0
114_0240_0560.JPG 6531 0
114_0320_0560.JPG 6531 0
114_0400_0560.JPG 6531 0
114_0480_0560.JPG 6531 0
114_0560_0560.JPG 6531 0
114_0640_0560.JPG 6531 0
114_0720_0560.JPG 6531 0
114_0800_0560.JPG 6531 0
114_0880_0560.JPG 6531 0
114_0960_0560.JPG 6531 0
114_1040_0560.JPG 6531 0


116_0400_0400.JPG 6667 2
116_0480_0400.JPG 6668 3
116_0560_0400.JPG 6669 1
116_0640_0400.JPG 6670 0
116_0720_0400.JPG 6670 0
116_0800_0400.JPG 6670 0
116_0880_0400.JPG 6670 0
116_0960_0400.JPG 6670 0
116_1040_0400.JPG 6670 1
116_1120_0400.JPG 6671 1
116_0000_0480.JPG 6672 2
116_0080_0480.JPG 6673 2
116_0160_0480.JPG 6674 0
116_0240_0480.JPG 6674 0
116_0320_0480.JPG 6674 0
116_0400_0480.JPG 6674 1
116_0480_0480.JPG 6675 2
116_0560_0480.JPG 6676 1
116_0640_0480.JPG 6677 0
116_0720_0480.JPG 6677 0
116_0800_0480.JPG 6677 0
116_0880_0480.JPG 6677 0
116_0960_0480.JPG 6677 0
116_1040_0480.JPG 6677 1
116_1120_0480.JPG 6678 1
116_0000_0560.JPG 6679 1
116_0080_0560.JPG 6680 1
116_0160_0560.JPG 6681 0
116_0240_0560.JPG 6681 0
116_0320_0560.JPG 6681 0
116_0400_0560.JPG 6681 1
116_0480_0560.JPG 6682 3
116_0560_0560.JPG 6683 2
116_0640_0560.JPG 6684 0
116_0720_0560.JPG 6684 0
116_0800_0560.JPG 6684 0
116_0880_0560.JPG 6684 0
116_0960_0560.JPG 6684 1
116_1040_0560.JPG 6685 2
116_1120_0560.JPG 6686 1


118_0560_0400.JPG 6777 0
118_0640_0400.JPG 6777 0
118_0720_0400.JPG 6777 0
118_0800_0400.JPG 6777 0
118_0880_0400.JPG 6777 0
118_0960_0400.JPG 6777 0
118_1040_0400.JPG 6777 0
118_1120_0400.JPG 6777 0
118_0000_0480.JPG 6777 0
118_0080_0480.JPG 6777 0
118_0160_0480.JPG 6777 0
118_0240_0480.JPG 6777 0
118_0320_0480.JPG 6777 0
118_0400_0480.JPG 6777 0
118_0480_0480.JPG 6777 0
118_0560_0480.JPG 6777 0
118_0640_0480.JPG 6777 0
118_0720_0480.JPG 6777 0
118_0800_0480.JPG 6777 0
118_0880_0480.JPG 6777 0
118_0960_0480.JPG 6777 0
118_1040_0480.JPG 6777 0
118_1120_0480.JPG 6777 0
118_0000_0560.JPG 6777 0
118_0080_0560.JPG 6777 0
118_0160_0560.JPG 6777 0
118_0240_0560.JPG 6777 0
118_0320_0560.JPG 6777 0
118_0400_0560.JPG 6777 0
118_0480_0560.JPG 6777 0
118_0560_0560.JPG 6777 0
118_0640_0560.JPG 6777 0
118_0720_0560.JPG 6777 0
118_0800_0560.JPG 6777 0
118_0880_0560.JPG 6777 0
118_0960_0560.JPG 6777 0
118_1040_0560.JPG 6777 0
118_1120_0560.JPG 6777 0
118_0000_0640.JPG 6777 0
118_0080_0640.JPG 6777 0


51_0720_0400.JPG 6904 0
51_0800_0400.JPG 6904 0
51_0880_0400.JPG 6904 0
51_0960_0400.JPG 6904 0
51_1040_0400.JPG 6904 0
51_1120_0400.JPG 6904 0
51_0000_0480.JPG 6904 0
51_0080_0480.JPG 6904 0
51_0160_0480.JPG 6904 0
51_0240_0480.JPG 6904 0
51_0320_0480.JPG 6904 0
51_0400_0480.JPG 6904 0
51_0480_0480.JPG 6904 0
51_0560_0480.JPG 6904 0
51_0640_0480.JPG 6904 0
51_0720_0480.JPG 6904 0
51_0800_0480.JPG 6904 0
51_0880_0480.JPG 6904 0
51_0960_0480.JPG 6904 0
51_1040_0480.JPG 6904 0
51_1120_0480.JPG 6904 0
51_0000_0560.JPG 6904 0
51_0080_0560.JPG 6904 0
51_0160_0560.JPG 6904 0
51_0240_0560.JPG 6904 0
51_0320_0560.JPG 6904 0
51_0400_0560.JPG 6904 0
51_0480_0560.JPG 6904 0
51_0560_0560.JPG 6904 0
51_0640_0560.JPG 6904 0
51_0720_0560.JPG 6904 0
51_0800_0560.JPG 6904 0
51_0880_0560.JPG 6904 0
51_0960_0560.JPG 6904 0
51_1040_0560.JPG 6904 0
51_1120_0560.JPG 6904 0
51_0000_0640.JPG 6904 0
51_0080_0640.JPG 6904 0
51_0160_0640.JPG 6904 0
51_0240_0640.JPG 6904 0
51_0320_0640.JPG 6904 0
51_0400_0640.JPG

In [11]:
s_o_files_id_map

6904

In [12]:
s_files_count

NameError: name 's_files_count' is not defined

In [13]:
for mapping, file_list in s_o_files_id_map.items():
    print(mapping, file_list)
    if mapping > 50:
        break

1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
2 [131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142]
3 [143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216]
4 [217, 218, 219, 2

In [14]:
# Put back
o_files_dictionary = files_dictionary
files_dictionary = s_files_dictionary

o_files_info_dictionary = files_info_dictionary
files_info_dictionary = s_files_info_dictionary

o_files_inverse_dictionary = files_inverse_dictionary
files_inverse_dictionary = s_files_inverse_dictionary
# s_label_name_dictionary = {}

o_labels_dictionary = labels_dictionary
labels_dictionary = s_labels_dictionary

o_boundingboxes_dictionary = boundingboxes_dictionary
boundingboxes_dictionary = s_boundingboxes_dictionary

In [15]:
print(files_dictionary)

{1: '1_0080_0000.JPG', 2: '1_0160_0000.JPG', 3: '1_0480_0000.JPG', 4: '1_0560_0000.JPG', 5: '1_0640_0000.JPG', 6: '1_0720_0000.JPG', 7: '1_0800_0000.JPG', 8: '1_0080_0080.JPG', 9: '1_0160_0080.JPG', 10: '1_0320_0080.JPG', 11: '1_0400_0080.JPG', 12: '1_0560_0080.JPG', 13: '1_0640_0080.JPG', 14: '1_0720_0080.JPG', 15: '1_0800_0080.JPG', 16: '1_1120_0080.JPG', 17: '1_0160_0160.JPG', 18: '1_0240_0160.JPG', 19: '1_0320_0160.JPG', 20: '1_0400_0160.JPG', 21: '1_0560_0160.JPG', 22: '1_0640_0160.JPG', 23: '1_0720_0160.JPG', 24: '1_0800_0160.JPG', 25: '1_0960_0160.JPG', 26: '1_1040_0160.JPG', 27: '1_1120_0160.JPG', 28: '1_0000_0240.JPG', 29: '1_0160_0240.JPG', 30: '1_0240_0240.JPG', 31: '1_0320_0240.JPG', 32: '1_0560_0240.JPG', 33: '1_0640_0240.JPG', 34: '1_0720_0240.JPG', 35: '1_0800_0240.JPG', 36: '1_0880_0240.JPG', 37: '1_0960_0240.JPG', 38: '1_1040_0240.JPG', 39: '1_1120_0240.JPG', 40: '1_0000_0320.JPG', 41: '1_0080_0320.JPG', 42: '1_0480_0320.JPG', 43: '1_0560_0320.JPG', 44: '1_0720_0320.JP

In [16]:
print(boundingboxes_dictionary)

defaultdict(<class 'list'>, {1: [{'x_coor': 110, 'y_coor': 89, 'width': 13, 'height': 22, 'label': 2}, {'x_coor': 133, 'y_coor': 74, 'width': 16, 'height': 11, 'label': 2}], 2: [{'x_coor': 30, 'y_coor': 89, 'width': 13, 'height': 22, 'label': 2}, {'x_coor': 53, 'y_coor': 74, 'width': 16, 'height': 11, 'label': 2}], 3: [{'x_coor': 120, 'y_coor': 68, 'width': 14, 'height': 18, 'label': 2}], 4: [{'x_coor': 40, 'y_coor': 68, 'width': 14, 'height': 18, 'label': 2}, {'x_coor': 122, 'y_coor': 45, 'width': 15, 'height': 10, 'label': 2}, {'x_coor': 140, 'y_coor': 73, 'width': 13, 'height': 17, 'label': 2}], 5: [{'x_coor': 42, 'y_coor': 45, 'width': 15, 'height': 10, 'label': 2}, {'x_coor': 60, 'y_coor': 73, 'width': 13, 'height': 17, 'label': 2}], 6: [{'x_coor': 138, 'y_coor': 35, 'width': 17, 'height': 19, 'label': 2}], 7: [{'x_coor': 58, 'y_coor': 35, 'width': 17, 'height': 19, 'label': 2}], 8: [{'x_coor': 110, 'y_coor': 9, 'width': 13, 'height': 22, 'label': 2}], 9: [{'x_coor': 30, 'y_coor':

# Label Merge *

In [12]:
print(label_name_dictionary)

{1: {'name': 'Lactobacillus    '}, 2: {'name': 'Gardnerella      '}, 3: {'name': 'Curved Rod       '}, 4: {'name': 'Unknown A        '}, 5: {'name': 'Coccus           '}, 6: {'name': 'Yeast            '}, 7: {'name': 'Noise - Edge     '}, 8: {'name': 'Noise - Decompose'}}


In [13]:
label_map_dict = {1: 1, 2: 2, 3: 3, 4: 6, 5: 4, 6: 5, 7: 6, 8: 6}
label_name_dict = {1: "Lactobacillus", 2: "Gardnerella", 3: "Curved Rods", 4: "Coccus", 5:"Yeast", 6: "Noise"}
noise_label = 6
for k, v in label_map_dict.items():
    print(k, label_name_dictionary[k], "-->", v, label_name_dict[v])

1 {'name': 'Lactobacillus    '} --> 1 Lactobacillus
2 {'name': 'Gardnerella      '} --> 2 Gardnerella
3 {'name': 'Curved Rod       '} --> 3 Curved Rods
4 {'name': 'Unknown A        '} --> 6 Noise
5 {'name': 'Coccus           '} --> 4 Coccus
6 {'name': 'Yeast            '} --> 5 Yeast
7 {'name': 'Noise - Edge     '} --> 6 Noise
8 {'name': 'Noise - Decompose'} --> 6 Noise


In [16]:
image_suffix = '.JPG'

dataset_prefix = '../data/dataset/'
dataset_suffix = '_20180411_original.record'

data_lists = {}
data_lists['dev'] = [7, 12, 19, 24, 39, 47, 68, 76, 86, 100, 115]
data_lists['test'] = [8, 20, 49, 67, 77, 85, 91, 117, 119]

data_lists['dev'] = [o_files_inverse_dictionary[str(i) + image_suffix] for i in data_lists['dev']]
data_lists['test'] = [o_files_inverse_dictionary[str(i) + image_suffix] for i in data_lists['test']]

data_lists['train'] = [i for i in range(1, 120) if i not in data_lists['dev'] and i not in data_lists['test']]
print(len(data_lists['train']), data_lists['train'])
print(len(data_lists['dev']), data_lists['dev'])
print(len(data_lists['test']), data_lists['test'])



99 [1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 14, 15, 16, 17, 18, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 67, 68, 69, 70, 71, 72, 73, 76, 77, 78, 79, 80, 81, 82, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 116, 118, 119]
11 [7, 12, 19, 24, 39, 47, 66, 74, 84, 98, 113]
9 [8, 20, 49, 65, 75, 83, 89, 115, 117]


In [17]:
def create_tf_record_single_file(file_id, verbose = False):
    if verbose:
        print("Processing:", file_id)
#     file_id = files_inverse_dictionary[file_name]
    path = os.path.abspath(files_info_dictionary[file_id]['path'])
    with tf.gfile.GFile(path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = files_info_dictionary[file_id]['name'].encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for row in boundingboxes_dictionary[file_id]:
        xmins.append(row['x_coor'] / width)
        xmaxs.append( (row['x_coor'] + row['width']) / width)
        ymins.append(row['y_coor'] / height)
        ymaxs.append((row['y_coor'] + row['height']) / height)
        label_id = label_map_dict[row['label']]
        classes_text.append(label_name_dict[label_id].encode('utf8'))
        classes.append(label_id)
        
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    if verbose:
        print("Finished processing ", os.path.basename(path), ", with number of labels = " + str(len(classes)))
    return tf_example

def create_dataset(file_id_list, output_path, verbose = False):
    writer = tf.python_io.TFRecordWriter(output_path)
    for file_id in file_id_list:
        tf_example = create_tf_record_single_file(file_id, verbose)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print("Finished writing dataset to " + output_path)

In [18]:
for stage in data_lists:

    print("Preparing data file for " + stage)
#     create_dataset([str(i) + image_suffix for i in data_lists[stage]], dataset_prefix + stage + dataset_suffix, True)
    if partial_dataset:
        data_nested_list = [s_o_files_id_map[i] for i in data_lists[stage]]
        flattened = [val for sublist in data_nested_list for val in sublist]
    else:
        flattened = data_lists[stage]
    create_dataset(flattened, dataset_prefix + stage + dataset_suffix, True)
    print("Total file count: " + str(len(flattened)))
    print("-------------------------------------------------------------------------------")

Preparing data file for dev
Processing: 7
Finished processing  7.JPG , with number of labels = 52
Processing: 12
Finished processing  12.JPG , with number of labels = 19
Processing: 19
Finished processing  19.JPG , with number of labels = 6
Processing: 24
Finished processing  24.JPG , with number of labels = 11
Processing: 39
Finished processing  39.JPG , with number of labels = 79
Processing: 47
Finished processing  47.JPG , with number of labels = 1
Processing: 66
Finished processing  68.JPG , with number of labels = 39
Processing: 74
Finished processing  76.JPG , with number of labels = 26
Processing: 84
Finished processing  86.JPG , with number of labels = 26
Processing: 98
Finished processing  100.JPG , with number of labels = 11
Processing: 113
Finished processing  115.JPG , with number of labels = 34
Finished writing dataset to ../data/dataset/dev_20180411_original.record
Total file count: 11
-------------------------------------------------------------------------------
Prepari

Finished writing dataset to ../data/dataset/train_20180411_original.record
Total file count: 99
-------------------------------------------------------------------------------


# Label Map

In [40]:
dataset_prefix = '../data/dataset/'
label_map_file_name = 'label_map.pbtxt'

In [41]:
with open(dataset_prefix + label_map_file_name,"w+") as f:
    for key, value in label_name_dict.items():
        print(key, value)
        f.write("item {{\n  id: {}\n  name: '{}'\n}}\n\n".format(key, value))

1 Lactobacillus
2 Gardnerella
3 Curved Rods
4 Coccus
5 Yeast
6 Noise


## Cloud Dataset 

In [ ]:
import base64
import io
import json
from PIL import Image
import tensorflow as tf

width = 1024
height = 768
predict_instance_json = "inputs.json"
predict_instance_tfr = "inputs.tfr"
with tf.python_io.TFRecordWriter(predict_instance_tfr) as tfr_writer:
    with open(predict_instance_json, "wb") as fp:
        for image in ["image1.jpg", "image2.jpg"]:
            img = Image.open(image)
            img = img.resize((width, height), Image.ANTIALIAS)
            output_str = io.BytesIO()
            img.save(output_str, "JPEG")
            fp.write(
              json.dumps({"b64": base64.b64encode(output_str.getvalue())}) + "\n")
            tfr_writer.write(output_str.getvalue())
            output_str.close()

# Dataset Inspection

In [49]:
import tensorflow as tf
from google.protobuf.json_format import MessageToJson

file = "../data/dataset/test_20180410.record"
fileNum=1
for string_record in tf.python_io.tf_record_iterator(file):
    example = tf.train.Example()
    example.ParseFromString(string_record)
    filename = (example.features.feature['image/filename'].bytes_list.value[0])
    bbox_xmin = (example.features.feature['image/object/bbox/xmin'].float_list.value)
    bbox_xmax = (example.features.feature['image/object/bbox/xmax'].float_list.value)
    bbox_ymin = (example.features.feature['image/object/bbox/ymin'].float_list.value)
    bbox_ymax = (example.features.feature['image/object/bbox/ymax'].float_list.value)

    print(filename)
    print(bbox_xmin)
    print(bbox_xmax)
    print(bbox_ymin)
    print(bbox_ymax)

    break

b'8_0000_0000.JPG'
[0.15000000596046448, 0.7749999761581421, 0.893750011920929]
[0.21250000596046448, 0.84375, 0.96875]
[0.518750011920929, 0.8500000238418579, 0.7875000238418579]
[0.5874999761581421, 0.981249988079071, 0.8687499761581421]


In [ ]:
[{'x_coor': 24, 'y_coor': 83, 'width': 10, 'height': 11, 'label': 2}, 
 {'x_coor': 124, 'y_coor': 136, 'width': 11, 'height': 21, 'label': 3}, 
 {'x_coor': 143, 'y_coor': 126, 'width': 12, 'height': 13, 'label': 2}]